In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
from sklearn.model_selection import train_test_split
from wordbatch.extractors import WordHash, WordBag
from wordbatch.models import FM_FTRL,FTRL
from sklearn.metrics import f1_score
import wordbatch
from keras.utils import np_utils
from sklearn.metrics import f1_score
import pickle
from scipy.sparse import csr_matrix, hstack

/usr/local/lib/python3.5/dist-packages/randomstate-1.14.0-py3.5-linux-x86_64.egg/randomstate/__init__.py:66: RandomStateDeprecationWarning: 
**End-of-life notification**

This library was designed to bring alternative generators to the NumPy 
infrastructure. It as been successful in advancing the conversation 
for a future implementation of a new random number API in NumPy which 
will allow new algorithms and/or generators. The next step
in this process is to separate the basic (or core RNG) from the 
functions that transform random bits into useful random numbers.
This has been implemented in a successor project  **randomgen** 
available on GitHub

https://github.com/bashtage/randomgen

or PyPi

https://pypi.org/project/randomstate/.

randomgen has a slightly different API, so please see the randomgen documentation

https://bashtage.github.io/randomgen.

  warnings.warn(DEPRECATION_MESSAGE, RandomStateDeprecationWarning)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: Futu

In [2]:
t1=time.time()
train = pd.read_csv('../input/train2.csv')
test = pd.read_csv('../input/test2.csv')
test_id = pd.read_csv('../input/new_data/test_set.csv')[["id"]].copy()
y=(train["class"]-1).astype(int)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
merge = pd.concat([train, test])

In [5]:
column = "word_seg"
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
new=False

In [51]:
# from scipy.sparse import hstack
from scipy import sparse
# trn_term_doc = sparse.load_npz('../feature/text_tifi/trn_term_doc.npz')
# trn_term_word = sparse.load_npz('../feature/text_tifi/trn_term_word.npz')
# trn_term_word_three = sparse.load_npz('../feature/text_tifi/trn_term_word_three.npz')
# train_feaure =  hstack([trn_term_doc,trn_term_word,trn_term_word_three]).tocsr()
# del trn_term_doc,trn_term_word,trn_term_word_three
# import gc
# gc.collect()
# # trn_term_word_four = sparse.load_npz('../feature/text_tifi/trn_term_word_four.npz')
# # trn_term_word_five = sparse.load_npz('../feature/text_tifi/trn_term_word_five.npz')
# trn_char = sparse.load_npz('../feature/text_tifi/trn_char.npz')
# docLen  =pd.DataFrame( np.array(train[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word = pd.DataFrame( np.array(train['article'].map(lambda x : len(x.split(" ")))/train[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))
# docLen_test  =pd.DataFrame( np.array(test[column].map(lambda x : len(x.split(" ")))/39759).reshape(-1,1))
# doclen_word_test = pd.DataFrame( np.array(test['article'].map(lambda x : len(x.split(" ")))/test[column].map(lambda x : len(x.split(" ")))).reshape(-1,1))

In [52]:
# X_train =  hstack([train_feaure, trn_char,doclen_word]).tocsr()
# del trn_char,doclen_word
# gc.collect()

In [53]:
# test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')
# test_term_word = sparse.load_npz('../feature/text_tifi/test_term_word.npz')
# test_term_word_three = sparse.load_npz('../feature/text_tifi/test_term_word_three.npz')
# # test_term_word_four = sparse.load_npz('../feature/text_tifi/test_term_word_four.npz')
# # test_term_word_five = sparse.load_npz('../feature/text_tifi/test_term_word_five.npz')
# test_char = sparse.load_npz('../feature/text_tifi/test_char.npz')
# X_test =  hstack([test_term_doc,test_term_word,test_term_word_three,test_char,doclen_word_test]).tocsr()
# del test_term_doc,test_term_word,test_term_word_three,test_char,doclen_word_test
# gc.collect()

In [72]:

# new=True
# if new:
#     trn_term_doc, test_term_doc= pickle.load(open('../feature/tifi_min4.pkl','rb'))
    
#     column="article"
#     n = train.shape[0]
#     vec = TfidfVectorizer(ngram_range=(1,2),min_df=4, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1, max_features=60000)
#     vec.fit(merge[column])
#     trn_article_doc = vec.transform(train[column])
#     test_article_doc = vec.transform(test[column])
# else:
#     trn_term_doc, test_term_doc= pickle.load(open('../feature/tifi_min4.pkl','rb'))
#     train_svd, test_svd= pickle.load(open('../feature/svd_5.pkl','rb'))
import pickle
svd_dir = '../feature/svd_5.pkl'
tifi_dir = '../feature/tifi_min4.pkl'
train_svd, test_svd= pickle.load(open(svd_dir,'rb'))
trn_term_doc, test_term_doc= pickle.load(open(tifi_dir,'rb'))
lda1 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature10.csv')
lda2 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature20.csv')
lda3 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature30.csv')
lda4 = pd.read_csv('../feature/other_features/features-vinson/ldaFeature40.csv')
test_term_doc = sparse.load_npz('../feature/text_tifi/test_term_doc.npz')
trn_term_doc = sparse.load_npz('../feature/text_tifi/trn_term_doc.npz')
train_feaure = hstack((trn_term_doc, train_svd, lda1.iloc[:102277,:], lda2.iloc[:102277,:], lda3.iloc[:102277,:], lda4.iloc[:102277,:], trn_term_doc)).tocsr()
test_festure = hstack((test_term_doc, test_svd, lda1.iloc[102277:,:], lda2.iloc[102277:,:], lda3.iloc[102277:,:], lda4.iloc[102277:,:], test_term_doc)).tocsr()

In [73]:
# w2v =  pd.read_csv('../feature/other_features/features-vinson/w2vFeatureSum100.csv')

In [74]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [81]:

name = ["class_prob_%s"%i for i in range(1,20)]

In [82]:
# X_train = hstack((trn_term_doc,trn_article_doc)).tocsr()
# X_test = hstack((test_term_doc,test_article_doc)).tocsr()
# X_train = trn_term_doc
# X_test = test_term_doc

In [83]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train_feaure, y, test_size=0.1, random_state=42)

In [84]:
test_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])
train_prob = pd.DataFrame([],columns=["class_prob_%s"%i for i in range(1,20)])

In [85]:
one_y_train=np_utils.to_categorical(Y_train,num_classes=19)
one_y_val=np_utils.to_categorical(Y_valid,num_classes=19)

In [87]:
from tqdm import tqdm
for i in tqdm(range(19)):
    model = FTRL(alpha=0.01, beta=0.00001, L1=0.00001, L2=0.1, D=X_train.shape[1],iters=17, inv_link="identity", threads=4)

    model.fit(X_train, one_y_train[:,i])
    valid_re = model.predict(X_valid)
    score= self_f1_score(one_y_val[:,i],valid_re)
    test_re = model.predict(test_festure)
    print ('score is {}'.format(score))
    test_prob['class_prob_%s'%(i+1)] = test_re
    train_prob['class_prob_%s'%(i+1)] = valid_re


  0%|          | 0/19 [00:00<?, ?it/s]

Total e: 8903.751843895627
Total e: 6657.294493113979
Total e: 5977.876348539086



Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm-4.19.9-py3.5.egg/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Total e: 5535.910989529965
Total e: 5200.870585188941
Total e: 4926.599636683414
Total e: 4692.729145546836
Total e: 4488.013819137346
Total e: 4305.346867231085
Total e: 4140.1525632440835
Total e: 3989.3768286064314
Total e: 3850.582714843816
Total e: 3722.011619081804
Total e: 3602.3379877044576
Total e: 3490.499483810601
Total e: 3385.509192393767
Total e: 3286.741619660799


  5%|▌         | 1/19 [00:32<09:45, 32.53s/it]

...

score is 0.8373656015583923
Total e: 7384.145934109973
Total e: 4945.751600205078
Total e: 4231.576795738725
Total e: 3782.2761604712096
Total e: 3449.782362896692
Total e: 3183.752388734104
Total e: 2962.2075872785995
Total e: 2772.6066664444234
Total e: 2606.6883593172583
Total e: 2459.3481093634987
Total e: 2327.131168702071
Total e: 2207.390450185072
Total e: 2098.1631848877273
Total e: 1997.9835372277548
Total e: 1905.689455770919
Total e: 1820.3620289879152
Total e: 1741.1375021809852


 95%|█████████▍| 18/19 [09:42<00:32, 32.34s/it]

score is 0.9210848461517627
Total e: 9349.163931893916
Total e: 6931.052457016191
Total e: 6190.4270161802515
Total e: 5707.484998164389
Total e: 5341.600747579082
Total e: 5043.81433527241
Total e: 4791.238629459333
Total e: 4570.893687498155
Total e: 4374.975110510681
Total e: 4198.320583831216
Total e: 4037.3605886140094
Total e: 3889.480370879817
Total e: 3752.7280078505814
Total e: 3625.5471483101055
Total e: 3506.728358253368
Total e: 3395.3852650076565
Total e: 3290.6557581349043


100%|██████████| 19/19 [10:13<00:00, 32.29s/it]

score is 0.7579755832487818


In [88]:
# predsF = model.predict(X_valid)
test_prob

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19
0,-0.173128,-0.063565,0.054936,-0.011877,0.948202,-0.060735,0.108552,-0.026035,-0.039620,-0.113206,0.106501,0.000479,-0.099414,0.041845,0.026922,-0.009735,0.083742,-0.056628,0.133344
1,0.128328,0.113261,-0.081516,0.857636,0.022486,0.040403,0.163758,-0.031026,0.035289,-0.024507,0.004297,0.098846,0.069572,-0.027888,0.028881,-0.050634,0.100988,0.162982,-0.055175
2,0.024923,0.058690,0.314154,0.051409,0.196079,0.027298,0.097038,0.264344,0.072324,-0.012945,0.106980,-0.033750,0.569538,-0.112994,0.045319,-0.116708,0.053012,-0.094663,-0.051998
3,0.133813,-0.040595,0.053339,1.067223,0.066010,0.038841,-0.042352,-0.015916,0.014076,-0.141235,0.073400,-0.185181,-0.006987,0.063131,-0.050674,-0.011817,0.004054,-0.094633,-0.013064
4,0.136678,-0.065823,0.118526,0.007860,0.731568,-0.030859,-0.018146,-0.136243,-0.070825,-0.054233,0.067191,0.074041,0.016746,0.060017,-0.056643,0.035033,-0.030330,0.013997,0.023247
5,0.017343,0.027029,0.052358,0.171816,0.555568,0.202583,-0.007467,0.111399,0.088145,0.107867,0.141298,0.086170,0.036536,-0.036134,0.025649,0.240326,-0.080593,0.095051,0.083800
6,0.115720,-0.047655,0.134141,0.134505,-0.077408,-0.029001,0.037086,-0.031747,-0.030105,0.130712,-0.069382,0.133033,-0.110612,-0.059474,0.615905,0.005436,0.094473,-0.037362,0.065623
7,-0.039557,-0.005291,0.156489,0.126175,0.075805,-0.032511,0.080510,-0.020075,-0.038383,-0.022238,0.014658,0.004430,0.059360,-0.099073,-0.015662,0.062906,0.385760,-0.115952,0.447749
8,-0.009995,0.014495,1.011273,-0.023888,-0.002773,-0.019820,-0.003480,0.022765,0.005601,-0.000922,-0.040258,-0.009388,0.008343,-0.005112,0.016257,0.020939,-0.031735,0.024991,-0.021851
9,0.281047,0.035421,-0.144076,0.056203,-0.031550,-0.002657,0.172813,0.143481,0.115273,0.148031,0.067583,0.423639,0.043996,-0.044357,-0.134227,0.070521,-0.119551,0.163858,0.192317


In [89]:
val_result = np.argmax(train_prob[name].values,axis=1)
print ('score is {}'.format(f1_score(Y_valid, val_result, average='macro')))

score is 0.7634721940633523


In [14]:
with open('../feature/stackint_pkl_file/fm22.pkl','wb') as fout:
    pickle.dump([train_prob,test_prob],fout)

In [90]:
test_prob["id"]=list(test_id["id"])
test_prob.to_csv('../output/prob_fm_baseline3.csv',index=None)

In [28]:
c = ["class_prob_%s"%i for i in range(1,21)]
preds=np.argmax(test_prob[c].values,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('../output/fm_baseline3.csv',index=None)

(102277, 1)
(102277, 1)


In [21]:
preds

array([ 5, 21, 21, ..., 21, 21, 21])

In [27]:
np.argmax(test_prob[c].values,axis=1)

array([ 4,  3, 12, ...,  5, 13, 12])

In [91]:
test_prob

,class_prob_1,class_prob_2,class_prob_3,class_prob_4,class_prob_5,class_prob_6,class_prob_7,class_prob_8,class_prob_9,class_prob_10,class_prob_11,class_prob_12,class_prob_13,class_prob_14,class_prob_15,class_prob_16,class_prob_17,class_prob_18,class_prob_19,id
0,-0.173128,-0.063565,0.054936,-0.011877,0.948202,-0.060735,0.108552,-0.026035,-0.039620,-0.113206,0.106501,0.000479,-0.099414,0.041845,0.026922,-0.009735,0.083742,-0.056628,0.133344,0
1,0.128328,0.113261,-0.081516,0.857636,0.022486,0.040403,0.163758,-0.031026,0.035289,-0.024507,0.004297,0.098846,0.069572,-0.027888,0.028881,-0.050634,0.100988,0.162982,-0.055175,1
2,0.024923,0.058690,0.314154,0.051409,0.196079,0.027298,0.097038,0.264344,0.072324,-0.012945,0.106980,-0.033750,0.569538,-0.112994,0.045319,-0.116708,0.053012,-0.094663,-0.051998,2
3,0.133813,-0.040595,0.053339,1.067223,0.066010,0.038841,-0.042352,-0.015916,0.014076,-0.141235,0.073400,-0.185181,-0.006987,0.063131,-0.050674,-0.011817,0.004054,-0.094633,-0.013064,3
4,0.136678,-0.065823,0.118526,0.007860,0.731568,-0.030859,-0.018146,-0.136243,-0.070825,-0.054233,0.067191,0.074041,0.016746,0.060017,-0.056643,0.035033,-0.030330,0.013997,0.023247,4
5,0.017343,0.027029,0.052358,0.171816,0.555568,0.202583,-0.007467,0.111399,0.088145,0.107867,0.141298,0.086170,0.036536,-0.036134,0.025649,0.240326,-0.080593,0.095051,0.083800,5
6,0.115720,-0.047655,0.134141,0.134505,-0.077408,-0.029001,0.037086,-0.031747,-0.030105,0.130712,-0.069382,0.133033,-0.110612,-0.059474,0.615905,0.005436,0.094473,-0.037362,0.065623,6
7,-0.039557,-0.005291,0.156489,0.126175,0.075805,-0.032511,0.080510,-0.020075,-0.038383,-0.022238,0.014658,0.004430,0.059360,-0.099073,-0.015662,0.062906,0.385760,-0.115952,0.447749,7
8,-0.009995,0.014495,1.011273,-0.023888,-0.002773,-0.019820,-0.003480,0.022765,0.005601,-0.000922,-0.040258,-0.009388,0.008343,-0.005112,0.016257,0.020939,-0.031735,0.024991,-0.021851,8
9,0.281047,0.035421,-0.144076,0.056203,-0.031550,-0.002657,0.172813,0.143481,0.115273,0.148031,0.067583,0.423639,0.043996,-0.044357,-0.134227,0.070521,-0.119551,0.163858,0.192317,9


In [24]:
c

['class_prob_1',
 'class_prob_2',
 'class_prob_3',
 'class_prob_4',
 'class_prob_5',
 'class_prob_6',
 'class_prob_7',
 'class_prob_8',
 'class_prob_9',
 'class_prob_10',
 'class_prob_11',
 'class_prob_12',
 'class_prob_13',
 'class_prob_14',
 'class_prob_15',
 'class_prob_16',
 'class_prob_17',
 'class_prob_18',
 'class_prob_19',
 'class_prob_20']